In [1]:
cd ..

/home/dmitriishubin/Desktop/Thesis/melanoma_experiment


In [16]:
import wfdb as wb
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from sklearn.model_selection import KFold
import cv2

# Get list of records

In [9]:
df =  pd.read_csv('../data/melanoma/train.csv')
image_name = df['image_name'].unique().tolist()
patients = df['patient_id'].unique().tolist()

len(patients)

2056

In [10]:
df

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0
33122,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0
33123,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0
33124,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0


In [5]:
len(image_name)

33126

In [34]:
for patient in patients:
    df_sub = df[df['patient_id'] == patient]['image_name'].tolist()
    
    os.makedirs(f'../data/melanoma/processed_data/{patient}',exist_ok=True)
    
    for image in df_sub:
        im= cv2.imread(f'../data/melanoma/jpeg/train/{image}.jpg')
        
        im = cv2.resize(im,(1024,1024))
        cv2.imwrite(f'../data/melanoma/processed_data/{patient}/{image}.jpg',im)

In [27]:
im.shape

(4000, 6000, 3)

In [14]:
df_sub

['ISIC_2637011',
 'ISIC_0078703',
 'ISIC_0092726',
 'ISIC_0179056',
 'ISIC_0229459',
 'ISIC_0234146',
 'ISIC_0262401',
 'ISIC_0281291',
 'ISIC_0304703',
 'ISIC_0324600',
 'ISIC_0448016',
 'ISIC_0673183',
 'ISIC_0881573',
 'ISIC_0924902',
 'ISIC_1216822',
 'ISIC_1408033',
 'ISIC_1426331',
 'ISIC_1448835',
 'ISIC_1481586',
 'ISIC_1495081',
 'ISIC_1575034',
 'ISIC_1577472',
 'ISIC_1614712',
 'ISIC_1786464',
 'ISIC_1805921',
 'ISIC_1858948',
 'ISIC_1902401',
 'ISIC_2061919',
 'ISIC_2284546',
 'ISIC_2287151',
 'ISIC_2332575',
 'ISIC_2399890',
 'ISIC_2524432',
 'ISIC_2598575',
 'ISIC_2753812',
 'ISIC_2760737',
 'ISIC_2801440',
 'ISIC_2828236',
 'ISIC_3046033',
 'ISIC_3392451',
 'ISIC_3393767',
 'ISIC_3490992',
 'ISIC_3537203',
 'ISIC_3584384',
 'ISIC_3789239',
 'ISIC_3870647',
 'ISIC_3932172',
 'ISIC_4327406',
 'ISIC_4455932',
 'ISIC_4547675',
 'ISIC_4564163',
 'ISIC_4730433',
 'ISIC_4787183',
 'ISIC_4861375',
 'ISIC_4862028',
 'ISIC_4901293',
 'ISIC_5063645',
 'ISIC_5212189',
 'ISIC_5252266

In [6]:
patients = np.array(patients)

kf = KFold(5,shuffle=True,random_state=42)
for fold,(train_index, test_index) in enumerate(kf.split(patients)):
    
    patients_train, patients_test = patients[train_index], patients[test_index]
    
    split = {
        'train': patients_train.tolist(),
        'val': patients_test.tolist(),
    }
    
    with open(f'./split_table/{fold}_split_table.json', 'w') as outfile:
            json.dump(split, outfile)